In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion(
    '1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.8.0


In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn(
        'No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    w1 = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    return w1, keep, layer3_out, layer4_out, layer7_out


tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [5]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    conv_1by1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same',
                                 kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    input = tf.layers.conv2d_transpose(conv_1by1, num_classes, 4, 2, padding='same',
                                       kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    pool_4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same',
                              kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    input = tf.add(input, pool_4)
    
    input = tf.layers.conv2d_transpose(input, num_classes, 4, 2, padding='same',
                                       kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    pool_3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same',
                              kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    input = tf.add(input, pool_3)
    output = tf.layers.conv2d_transpose(input, num_classes, 16, strides=(8, 8), padding='same',
                                        kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                        kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))

    return output


tests.test_layers(layers)

Tests Passed


In [6]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1,num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)  # 1e-2
    return logits, train_op, cross_entropy_loss


tests.test_optimize(optimize)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Tests Passed


In [11]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for image, label in get_batches_fn(batch_size):
            # train
            _, loss = sess.run([train_op, cross_entropy_loss],  feed_dict={
                     input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 0.0005})
            print("epoch{} Loss: = {:.3f}".format(epoch, loss))

tests.test_train_nn(train_nn)

epoch0 Loss: = 10.110
epoch0 Loss: = 10.110
Tests Passed


In [12]:
def run():
    num_classes = 2
    image_shape = (160, 576)  # KITTI dataset uses 160x576 images
    data_dir = './data'
    runs_dir = './runs'
    epochs = 10
    batch_size = 5

    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(
            os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        layers_output = layers(layer3_out, layer4_out, layer7_out, num_classes)
        logits, train_op, cross_entropy_loss = optimize(layers_output, correct_label, learning_rate, num_classes)
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video


# if __name__ == '__main__':
run()

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
epoch0 Loss: = 1.415
epoch0 Loss: = 21.909
epoch0 Loss: = 0.965
epoch0 Loss: = 0.733
epoch0 Loss: = 0.720
epoch0 Loss: = 0.701
epoch0 Loss: = 0.687
epoch0 Loss: = 0.684
epoch0 Loss: = 0.687
epoch0 Loss: = 0.677
epoch0 Loss: = 0.672
epoch0 Loss: = 0.671
epoch0 Loss: = 0.663
epoch0 Loss: = 0.662
epoch0 Loss: = 0.652
epoch0 Loss: = 0.641
epoch0 Loss: = 0.655
epoch0 Loss: = 0.606
epoch0 Loss: = 0.618
epoch0 Loss: = 0.587
epoch0 Loss: = 0.594
epoch0 Loss: = 0.564
epoch0 Loss: = 0.541
epoch0 Loss: = 0.534
epoch0 Loss: = 0.469
epoch0 Loss: = 0.514
epoch0 Loss: = 0.532
epoch0 Loss: = 0.431
epoch0 Loss: = 0.457
epoch0 Loss: = 0.410
epoch0 Loss: = 0.414
epoch0 Loss: = 0.387
epoch0 Loss: = 0.379
epoch0 Loss: = 0.367
epoch0 Loss: = 0.378
epoch0 Loss: = 0.287
epoch0 Loss: = 0.409
epoch0 Loss: = 0.318
epoch0 Loss: = 0.407
epoch0 Loss: = 0.437
epoch0 Loss: = 0.407
epoch0 Loss: = 0.353
epoch0 Loss: = 0.252
epoch0 

epoch6 Loss: = 0.096
epoch6 Loss: = 0.095
epoch6 Loss: = 0.104
epoch6 Loss: = 0.167
epoch6 Loss: = 0.093
epoch6 Loss: = 0.089
epoch6 Loss: = 0.134
epoch6 Loss: = 0.137
epoch6 Loss: = 0.105
epoch6 Loss: = 0.105
epoch6 Loss: = 0.080
epoch6 Loss: = 0.093
epoch6 Loss: = 0.092
epoch6 Loss: = 0.138
epoch6 Loss: = 0.134
epoch6 Loss: = 0.085
epoch6 Loss: = 0.107
epoch6 Loss: = 0.108
epoch6 Loss: = 0.144
epoch6 Loss: = 0.191
epoch7 Loss: = 0.084
epoch7 Loss: = 0.108
epoch7 Loss: = 0.147
epoch7 Loss: = 0.114
epoch7 Loss: = 0.103
epoch7 Loss: = 0.106
epoch7 Loss: = 0.094
epoch7 Loss: = 0.086
epoch7 Loss: = 0.119
epoch7 Loss: = 0.073
epoch7 Loss: = 0.204
epoch7 Loss: = 0.063
epoch7 Loss: = 0.114
epoch7 Loss: = 0.092
epoch7 Loss: = 0.080
epoch7 Loss: = 0.119
epoch7 Loss: = 0.094
epoch7 Loss: = 0.083
epoch7 Loss: = 0.119
epoch7 Loss: = 0.147
epoch7 Loss: = 0.102
epoch7 Loss: = 0.126
epoch7 Loss: = 0.179
epoch7 Loss: = 0.137
epoch7 Loss: = 0.176
epoch7 Loss: = 0.112
epoch7 Loss: = 0.180
epoch7 Loss: 